In [ ]:
# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)

plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [ ]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Download the data


In [ ]:
!gdown --id 1V6HsBoEczDoo4NDZ1I5iXSfRxFxCatis

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1V6HsBoEczDoo4NDZ1I5iXSfRxFxCatis
To: /content/1_구어체(1).xlsx
100% 15.8M/15.8M [00:00<00:00, 123MB/s]


In [ ]:
!gdown --id 1-Ar-wV5dhZAOALn-oa7uZ3eT6EKcepmh

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-Ar-wV5dhZAOALn-oa7uZ3eT6EKcepmh
To: /content/한영번역_타켓문장.csv
100% 7.25k/7.25k [00:00<00:00, 7.61MB/s]


In [ ]:
import pandas as pd

df_train = pd.read_excel('/content/1_구어체(1).xlsx')
df_train

,SID,원문,번역문
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2,씨티은행에서 일하세요?,Do you work at a City bank?
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
...,...,...,...
199995,199996,나는 먼저 청소기로 바닥을 밀었어요.,"First of all, I vacuumed the floor."
199996,199997,나는 먼저 팀 과제를 하고 놀러 갔어요.,I did the team assignment first and went out t...
199997,199998,나는 비 같은 멋진 연예인을 좋아해요.,I like cool entertainer like Rain.
199998,199999,나는 멋진 자연 경치를 보고 눈물을 흘렸어.,I cried seeing the amazing scenery.


In [ ]:
df_test = pd.read_csv('/content/한영번역_타켓문장.csv')
df_test

,한글
0,정리가 필요하겠네요.
1,처음 만났을 때를 떠올려 보세요
2,구썸남 인스타에 좋아요 누름.
3,있었던 일을 차분히 생각해봐요.
4,많이 찍다보면 조금씩 실력이 늘거예요.
...,...
195,조심히 오세요.
196,잘 이겨내고 있네요.
197,궁금할 수 있어요.
198,안 사귀는 것보다 좋지요.


## Modeling


Configuration

In [ ]:
batch_size = 128  # Batch size for training.
epochs = 50  # Number of epochs to train for.
hidden_units = 256  # Latent dimensionality of the encoding space.
embedding_dim = 64
num_samples = 200000  # Number of samples to train on.

Preprocessing

In [ ]:
def preprocess_sentence(sent):

  sent = sent.lower()

  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent


def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []


  for i in range(len(df_train)) :
    src_line = df_train['원문'][i]
    tar_line = df_train['번역문'][i]

  # source 데이터 전처리
    src_line = [w for w in preprocess_sentence(src_line).split()]
    src_line = src_line

    # target 데이터 전처리
    tar_line = preprocess_sentence(tar_line)
    tar_line_in = [w for w in ("<sos> " + tar_line).split()]  # teacher forcing을 위한 정답셋 
    tar_line_out = [w for w in (tar_line + " <eos>").split()]

    encoder_input.append(src_line)
    decoder_input.append(tar_line_in)
    decoder_target.append(tar_line_out)

    if i == num_samples - 1:
      break
              
  return encoder_input, decoder_input, decoder_target

In [ ]:
# 전처리 테스트
kor_sent = u"씨티은행에서 일하세요?" # u = utf-8 깨지지 않게
en_sent = u"Do you work at a City bank?"

print('전처리 전 한국어 문장 :', kor_sent)
print('전처리 후 한국어 문장 :',preprocess_sentence(kor_sent))
print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :', preprocess_sentence(en_sent))

전처리 전 한국어 문장 : 씨티은행에서 일하세요?
전처리 후 한국어 문장 : 씨티은행에서 일하세요 ?
전처리 전 영어 문장 : Do you work at a City bank?
전처리 후 영어 문장 : do you work at a city bank ?


In [ ]:
sents_kor_in, sents_en_in, sents_en_out  = load_preprocessed_data()

In [ ]:
print('인코더의 입력 :',sents_kor_in[:5])
print('디코더의 입력 :',sents_en_in[:5])
print('디코더의 레이블 :',sents_en_out[:5])

인코더의 입력 : [['.', '앱입니다', '컬러링', '있는', '수', '할', '체험', '이야기를', '아름다운', '성경의', '은', 'coloring', 'bible'], ['?', '일하세요', '씨티은행에서'], ['.', '기록하였다', '완판을', '차', '입소문만으로', '해외에서', '베스트셀러는', '푸리토의'], ['.', '살리셨습니다', '가운데서', '자', '죽은', '불러내어', '무덤에서', '나사로를', '이번엔', '예수님이', '장에서는'], ['.', '감사하겠습니다', '알려주시면', '제게', '될지', '재입고', '더', '개나', '몇', '사이즈가', '.']]
디코더의 입력 : [['<sos>', 'bible', 'coloring', 'is', 'a', 'coloring', 'application', 'that', 'allows', 'you', 'to', 'experience', 'beautiful', 'stories', 'in', 'the', 'bible', '.'], ['<sos>', 'do', 'you', 'work', 'at', 'a', 'city', 'bank', '?'], ['<sos>', 'purito', 's', 'bestseller', 'which', 'recorded', 'th', 'rough', 'cuts', 'by', 'words', 'of', 'mouth', 'from', 'abroad', '.'], ['<sos>', 'in', 'chapter', 'jesus', 'called', 'lazarus', 'from', 'the', 'tomb', 'and', 'raised', 'him', 'from', 'the', 'dead', '.'], ['<sos>', 'i', 'would', 'feel', 'grateful', 'to', 'know', 'how', 'many', 'stocks', 'will', 'be', 'secured', 'of', 'size', '.', 'and', '.']

단어집합 만들기

In [ ]:
# 한국어
tokenizer_kor = Tokenizer(filters="", lower=False)
tokenizer_kor.fit_on_texts(sents_kor_in)

# 인코더 데이터
encoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)

In [ ]:
# 영어
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
tokenizer_en.fit_on_texts(sents_en_out)

# 디코더 데이터
decoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
decoder_target = tokenizer_en.texts_to_sequences(sents_en_out)

Padding

In [ ]:
encoder_input = pad_sequences(encoder_input, padding='post')
decoder_input = pad_sequences(decoder_input, padding='post')
decoder_target = pad_sequences(decoder_target, padding='post')

In [ ]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (200000, 31)
디코더의 입력의 크기(shape) : (200000, 52)
디코더의 레이블의 크기(shape) : (200000, 52)


In [ ]:
src_vocab_size = len(tokenizer_kor.word_index) + 1
tar_vocab_size = len(tokenizer_en.word_index) + 1

print(f"한국어 단어 집합의 크기 : {src_vocab_size}, 영어 단어 집합의 크기 : {tar_vocab_size}")

한국어 단어 집합의 크기 : 194067, 영어 단어 집합의 크기 : 35242


In [ ]:
src_to_index = tokenizer_kor.word_index  # word : idx
index_to_src = tokenizer_kor.index_word  # idx : word
tar_to_index = tokenizer_en.word_index # word : idx
index_to_tar = tokenizer_en.index_word # idx : word

Random

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [168591 173630  88507 ... 120449 117962 144573]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

10% Data test

In [ ]:
n_of_val = int(num_samples*0.1)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (180000, 31)
훈련 target 데이터의 크기 : (180000, 52)
훈련 target 레이블의 크기 : (180000, 52)
테스트 source 데이터의 크기 : (20000, 31)
테스트 target 데이터의 크기 : (20000, 52)
테스트 target 레이블의 크기 : (20000, 52)


## Modeling

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

#### Encoder

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None, ))

# Embedding Layer
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)

# Return_state = True
# LSTM 층 2개 추가
encoder_lstm1 = LSTM(hidden_units, return_sequences=True, return_state = True)
encoder_lstm2 = LSTM(hidden_units, return_sequences=True, return_state = True)
encoder_lstm3 = LSTM(hidden_units, return_state = True)

# 은닉 상태와 셀 상태를 리턴
# encoder_outputs, state_h, state_c =  encoder_lstm(enc_emb)
x1 =  encoder_lstm1(enc_emb)
x2 = encoder_lstm2(x1)
encoder_outputs, state_h, state_c =  encoder_lstm3(x2)

# 인코더의 은닉 상태와 셀 상태를 저장
encoder_states = [state_h, state_c] 

#### Decoder

In [ ]:
from tensorflow.keras.layers import Attention

In [ ]:
# Decoder
decoder_inputs = Input(shape=(None, ))

# Embedding Layer
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)

# Embedding Result
dec_emb = dec_emb_layer(decoder_inputs)

# Return_state = True, Return_sequences = True
# LSTM 층 1개 추가
decoder_lstm1 = LSTM(hidden_units, return_sequences=True, return_state = True)
decoder_lstm2 = LSTM(hidden_units, return_sequences=True, return_state = True)

# initial_state = encoder_states
# decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = encoder_states)
x = decoder_lstm1(dec_emb, initial_state = encoder_states)
decoder_outputs, _, _ = decoder_lstm2(x)

# Attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1) # query 만들기

attention = Attention(hidden_units) # key, value 만들기
context_vector, _ = attention([ S_, encoder_outputs], return_attention_scores = True) 

concat = tf.concat([decoder_outputs, context_vector], axis=-1)

# Softmax
decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(concat)

In [ ]:
# Input / Output
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
# EarlyStopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, mode = 'auto')

In [ ]:
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto',
#     min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, 
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50, callbacks = [early_stopping])

Epoch 1/50
1407/1407 [==============================] - 377s 261ms/step - loss: 1.3236 - acc: 0.8130 - val_loss: 1.0705 - val_acc: 0.8370
Epoch 2/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.9337 - acc: 0.8548 - val_loss: 0.8265 - val_acc: 0.8721
Epoch 3/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.7197 - acc: 0.8843 - val_loss: 0.6678 - val_acc: 0.8937
Epoch 4/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.5616 - acc: 0.9056 - val_loss: 0.5430 - val_acc: 0.9111
Epoch 5/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.4422 - acc: 0.9222 - val_loss: 0.4534 - val_acc: 0.9257
Epoch 6/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.3533 - acc: 0.9349 - val_loss: 0.3915 - val_acc: 0.9338
Epoch 7/50
1407/1407 [==============================] - 365s 260ms/step - loss: 0.2858 - acc: 0.9448 - val_loss: 0.3538 - val_acc: 0.9399
Epoch 8/50
1407/1407 [============

#### Translator_Training

In [ ]:
# 인코더 - 훈련과 동일 
encoder_model = Model(encoder_inputs, [encoder_outputs, encoder_states])

In [ ]:
# 디코더 

# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

encoder_state_h = Input(shape=(hidden_units,))
encoder_outputs2 = Input(shape =(None, hidden_units,))


# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h, state_c = decoder_lstm2(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h, state_c]



# Attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs2[:, :-1, :]], axis=1) # query 

context_vector, att_score = attention([decoder_outputs2, encoder_outputs2], return_attention_scores = True)
decoder_concat = tf.concat([S_, context_vector], axis=-1)

# 모든 시점에 대해서 단어 예측 (Fully Connected)
decoder_outputs2 = decoder_dense(decoder_concat)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs, encoder_state_h, encoder_outputs2] + decoder_states_inputs,
    [decoder_outputs2, att_score] + decoder_states2)

In [ ]:
def decode_sequence(sentence):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  max_src_len, max_tar_len  = 15, 9

  attention_plot = np.zeros((max_tar_len, max_src_len))

  sentence = preprocess_sentence(sentence)

  inputs = [tokenizer_kor.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_src_len,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  encoder_test, states_value = encoder_model.predict(inputs)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  for t in range(max_tar_len):
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용                        ht
    output_tokens, att_score, h, c = decoder_model.predict([target_seq, states_value[0], encoder_test] + states_value)

    # 어텐션 가중치 시각화를 위해 저장 
    att_score = tf.reshape(att_score, (-1, ))
    attention_plot[t] = att_score.numpy() 

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char


    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if sampled_char == '<eos>':
        break 

  return sentence, decoded_sentence, attention_plot

학습과정에서의 정확도와 loss 히스토리를 시각화

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

# early stopping epoch 
epochs_range = history.epoch

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()